In [ ]:
# Import python tools
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
# old clip
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# Path to sqlite
database_path = "Resources/hawaii.sqlite"

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine(f"sqlite:///{database_path}")

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

In [ ]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# View the tables another way just for practice
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
# Save references to each table as variable
mea = Base.classes.measurement
sta = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Take a peek at the data
engine.execute('select * from measurement limit 5').fetchall()

In [ ]:
# get the column names and dtypes for measurement table
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Find the most recent date in the data set,
# & pass it into a variable.
most_recent_date = session.query(mea.date).order_by(mea.date.desc()).first()
most_recent_date = most_recent_date[0]
most_recent_date

### Design a query to retrieve the last 12 months of precipitation data and plot the results. 

In [ ]:
# Starting from the most recent data point in the database,
# Calculate the date one year from the last date in data set,
# Perform a query to retrieve the dates and precipitation values,
# & Save the query results, sorted by date, as a variable
prcpyear_df = session.query(mea.date, mea.prcp).filter(mea.date > '2016-08-22').order_by(mea.date).all()
prcpyear_df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
df = pd.DataFrame(prcpyear_df, columns=['date', 'precipitation'])
df.set_index('date', inplace=True)
df.plot(figsize=(9, 4))
plt.xlabel ("Date")
plt.ylabel ("Inches")
plt.xticks (rotation="vertical")
plt.savefig("Resources/Precipitation.png")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
pastyear_df = pd.DataFrame(session.query(mea.date, mea.prcp).filter(mea.date > '2016-08-22').order_by(mea.date))
pastyear_df = pastyear_df.set_index('date')
pastyear_df.describe()


# Exploratory Station Analysis

In [ ]:
# Take a peek at the data
engine.execute('select * from station').fetchall()

In [ ]:
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
# Design a query to find the total number of stations in the dataset
station_count = session.query(func.count(sta.station))[0][0]
station_count

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts
grouped_act = session.query(mea.station, func.count(mea.station)).group_by(mea.station).order_by(func.count(mea.station)).all()
grouped_act

##### Most Active Station ID USC00519281

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
temps = session.query(mea.station, func.min(mea.tobs), func.max(mea.tobs), func.avg(mea.tobs)).filter(mea.station == 'USC00519281').all()
temps

##### A better way to read that might look like this

In [ ]:
most_act_sta = session.query(mea.station, sta.name).filter(mea.station == 'USC00519281').distinct()[0]
min_temp = session.query(func.min(mea.tobs)).filter(mea.station == 'USC00519281').all()[0][0]
max_temp = session.query(func.max(mea.tobs)).filter(mea.station == 'USC00519281').all()[0][0]
avg_temp = session.query(func.avg(mea.tobs)).filter(mea.station == 'USC00519281').all()[0][0]
print(
    f"The most active station is '{most_act_sta}'\
    The lowest temperature recorded there was'{min_temp}'.\
    The highest temperature recorded there was'{max_temp}'.\
    The average temperature for all records is '{avg_temp}'.")

##### Or we could look at it again this way.

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
temps_df = pd.DataFrame(session.query(mea.station, mea.tobs).filter(mea.station == 'USC00519281'))
#temps_df = temps_df.set_index('date')
temps_df.describe()

In [ ]:
# Design a query to retrieve the previous 12 months of temperature observation data (TOBS).
tobsyear_df = session.query(mea.date, mea.tobs).filter(mea.date > '2016-08-22').order_by(mea.date).all()
tobsyear_df

In [ ]:
# Filter by the station with the highest number of observations.
# Query the previous 12 months of temperature observation data for only the most active station.
actobsyear_df = pd.DataFrame(session.query(mea.date, mea.tobs).filter(mea.station == 'USC00519281').filter(mea.date > '2016-08-22').order_by(mea.date))
actobsyear_df =actobsyear_df.set_index('date')
actobsyear_df.head()

In [ ]:
# Plot the results as a histogram with bins=12, as shown in the following image:
actobsyear_df['tobs'].hist(bins=12, figsize=(8.5, 5.5))
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.legend(['tobs'], )
plt.savefig("Resources/act_tobs_year.png")
plt.show()

# Close session

In [ ]:
# Close Session
session.close()